# Get Torchy's Tacos locations

#### Load Python tools and Jupyter config

In [1]:
%load_ext lab_black

In [2]:
import json
import requests
import pandas as pd
import altair as alt
import geopandas as gpd
from vega_datasets import data

In [3]:
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = None

In [4]:
place = "Torchy's Tacos"

## Read data

#### Get all the locations

In [5]:
headers = {
    "authority": "torchystacos.com",
    "accept": "*/*",
    "accept-language": "en-US,en;q=0.9,es;q=0.8",
    "content-type": "application/json; charset=UTF-8",
    "origin": "https://torchystacos.com",
    "sec-ch-ua": '"Not_A Brand";v="8", "Chromium";v="120", "Google Chrome";v="120"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"macOS"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "user-agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
}

#### This is too easy

In [6]:
response = requests.get(
    "https://torchystacos.com/locations-json/",
    headers=headers,
)

In [7]:
df = pd.DataFrame(response.json())[
    [
        "id",
        "name",
        "lat",
        "lon",
        "address",
        "city",
        "state_abbr",
        "zip",
        "phone",
    ]
].sort_values("lon", ascending=False)

--- 

## Process

#### Fill in missing values

In [9]:
df[df["lon"] == ""]

,id,name,lat,lon,address,city,state_abbr,zip,phone
9,3964,Midlothian,,,1315 Huguenot Road,Midlothian,VA,23113,
10,3963,Kyle,,,19100 IH-35,Kyle,TX,78640,


In [10]:
df.loc[df["name"] == "Midlothian", "lon"] = -77.61283589944435
df.loc[df["name"] == "Kyle", "lon"] = -97.84947150552404
df.loc[df["name"] == "Ronald Regan Leander", "lon"] = -97.82606689558637
df.loc[df["name"] == "Midlothian", "lat"] = 37.50993479610436
df.loc[df["name"] == "Kyle", "lat"] = 30.0304330834341
df.loc[df["name"] == "Ronald Regan Leander", "lat"] = 30.6361144773755

#### Clean up lon/lat

In [11]:
df["lon"] = df.lon.astype(float)
df["lat"] = df.lat.astype(float)

In [20]:
len(df)

124

---

## Geography

#### Make a geodataframe from lon/lat

In [12]:
df_geo = df.copy()

In [13]:
gdf = gpd.GeoDataFrame(
    df_geo, geometry=gpd.points_from_xy(df_geo.lon, df_geo.lat)
).set_crs("epsg:4326")

---

## Maps

#### US states background

In [14]:
background = (
    alt.Chart(alt.topo_feature(data.us_10m.url, feature="states"))
    .mark_geoshape(fill="#e9e9e9", stroke="white")
    .properties(width=800, height=500, title=f"{place} locations")
    .project("albersUsa")
)

#### Location points map

In [15]:
points = (
    alt.Chart(gdf)
    .mark_circle(size=10, color="red")
    .encode(
        longitude="lon:Q",
        latitude="lat:Q",
    )
)

point_map = background + points
point_map.configure_view(stroke=None)

alt.LayerChart(...)

#### Location proportional symbols map

In [16]:
symbols = (
    alt.Chart(gdf)
    .transform_aggregate(
        latitude="mean(lat)",
        longitude="mean(lon)",
        count="count()",
        groupby=["state_abbr"],
    )
    .mark_circle()
    .encode(
        longitude="longitude:Q",
        latitude="latitude:Q",
        size=alt.Size("count:Q", title="Count by state"),
        color=alt.value("red"),
        tooltip=["state:N", "count:Q"],
    )
    .properties(title=f"Number of {place} in US, by average lon/lat of locations")
)

symbol_map = background + symbols
symbol_map.configure_view(stroke=None)

alt.LayerChart(...)

---

## Exports

#### CSV

In [17]:
df.to_csv("data/processed/torchys_tacos_locations.csv", index=False)

#### JSON

In [18]:
df.to_json("data/processed/torchys_tacos_locations.json", indent=4, orient="records")

#### GeoJSON

In [19]:
gdf.to_file("data/processed/torchys_tacos_locations.geojson", driver="GeoJSON")